<a href="https://colab.research.google.com/github/AlessandroMirone/MBA_UkraineWar_2022/blob/main/mark2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

steps of the analysis - 1 - loading and preprocessing: \
In a distributed environment (Eg. Hadhoop) data have to be loaded into the distributed storage system. We assume this step is already carried out. Data would be in zip form to save space. \
after the data have been loaded, they are exported to the environments of each node with pd.read.csv(). The data are still saved as zipped file inside each node memory, but now they can be accessed.
we can prepare them for computation using spark by creating an RDD with parallelize. In this step, only english tweets are retained -> rdd = df.parallelize(df.text[df.language = 'english']) \
we can then transform this rdd as to obtain a single array splitted between the nodes, with positions in the array occupied by the texts of each tweet (ie baskets): a map function will take as inputs the 'text' column's values and the pd.Series function, stacking each text content in the positions of the array. The result will be an rdd containing an array of lists split between the different nodes. Each list contain a single element (the full text) \
we can use another map function with split in order to separate the lists into lists of single elements, each list of single elements now occupying the corresponding position in the array. \
we can then apply the cleaning functions to the array: each list (every list containing the sequence of words in the corresponding text) will be cleaned and lemmatized. The result is the same, an array of lists (baskets) each containing the tokens present in the relative basket as single elements. We can directly apply the a-priori algorithm to this array.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import numpy as np
import pandas as pd
import csv
import os

In [6]:
os.environ["KAGGLE_USERNAME"] = "alessandromirone"
os.environ["KAGGLE_KEY"] = "c881e45ce076cb815820c1e582302936"

!kaggle datasets download bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows -f 0905_UkraineCombinedTweetsDeduped.csv.gzip

0905_UkraineCombinedTweetsDeduped.csv.gzip.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
#spark context
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
spark

In [8]:
#load zipped file in environment
from zipfile import ZipFile
with ZipFile('0905_UkraineCombinedTweetsDeduped.csv.gzip.zip', 'r') as zip:
    zip.extractall()
filename = r'0905_UkraineCombinedTweetsDeduped.csv.gzip'
df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
df

,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,...,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts
0,567289542,Saudi_Gazette,Saudi Gazette is a leading English language da...,Saudi Arabia,2,435540,134609,2012-04-30 13:01:28.000000,1566576845415501824,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 06:22:59.378541
1,1244796162751655936,RussianLadies1,Russia Ladies is one of the largest and most r...,Russia,903,412,554,2020-03-31 01:19:00.000000,1566576846745092097,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 07:15:37.978958
2,161871799,ZacatecasImagen,El periódico líder en Zacatecas,"Zacatecas, México",586,23190,178163,2010-07-02 00:48:16.000000,1566576846795411459,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 11:31:20.980226
3,984429894829592576,pulsoguayaco,"🌐✈Blog de aviación, viajes y economía para via...",🇪🇨🇺🇸,75,324,14459,2018-04-12 13:55:51.000000,1566576847051264002,2022-09-05 00:00:00,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 11:31:20.967250
4,2445145434,Lyobserver,The Libya Observer is a key source for compreh...,Tripoli,96,77763,60461,2014-04-15 08:59:11.000000,1566576849035018240,2022-09-05 00:00:01,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-05 07:15:37.907896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44690,1532665429075935235,NicholasFreshne,"The views, information, or opinions expressed ...",USA,4,15,331,2022-06-03 10:08:29.000000,1566939152523972610,2022-09-05 23:59:40,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:40:18.561582
44691,1557539843219066882,Ludmila_Volkov,NaN,NaN,151,29,4241,2022-08-11 01:30:24.000000,1566939158182076417,2022-09-05 23:59:42,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:46:48.566650
44692,1565837550861828097,Tuchua2,#staywhitUkraine a small contribution to the e...,Leipzigngerd,89,579,1241,2022-09-02 23:02:57.000000,1566939194307612673,2022-09-05 23:59:50,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:46:48.556311
44693,31095945,shaancheema,I #Am #Zeeshan #Hussain #Equality #Justice (Be...,United Kingdom,114038,54654,193556,2009-04-14 10:34:44.000000,1566939215816007680,2022-09-05 23:59:56,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-09-06 01:46:48.546007


In [9]:
df.columns

Index(['userid', 'username', 'acctdesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreatedts', 'tweetid', 'tweetcreatedts',
       'retweetcount', 'text', 'hashtags', 'language', 'coordinates',
       'favorite_count', 'is_retweet', 'original_tweet_id',
       'original_tweet_userid', 'original_tweet_username',
       'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_screen_name', 'is_quote_status', 'quoted_status_id',
       'quoted_status_userid', 'quoted_status_username', 'extractedts'],
      dtype='object')

In [10]:
pd.unique(df.language) #many languages, only english texts will be considered

array(['en', 'es', 'ja', 'ar', 'no', 'uk', 'und', 'ru', 'zh', 'de', 'da',
       'it', 'vi', 'fr', 'pt', 'kn', 'ka', 'et', 'iw', 'in', 'lt', 'el',
       'fi', 'hi', 'is', 'th', 'nl', 'ca', 'eu', 'gu', 'pl', 'cy', 'tr',
       'bg', 'ro', 'lv', 'sv', 'ur', 'tl', 'sr', 'ta', 'ko', 'cs', 'ht',
       'fa', 'te', 'am', 'hu', 'bn', 'ml', 'mr', 'my', 'sl', 'ckb', 'ps',
       'or', 'pa', 'ne', 'hy'], dtype=object)

In [11]:
rdd = sc.parallelize(df.text[df.language == 'en'])
rdd.take(50)

['The Russian-controlled #Zaporizhzhia nuclear plant in #Ukraine has lost its link to its last remaining main power line, the #UN nuclear watchdog said.\nhttps://t.co/YrWSRtQmS7',
 'Russia is home to some of the best and most esteemed writers like Leo Tolstoy, Fyodor Dostoevsky, and Anton Chekhov,\n\nFollow us for more interesting facts about Russia!\nhttps://t.co/9gORMVnGIg\n\n#Russia #interesting #Facts #Literature #Truefacts #followusnow #FYP https://t.co/k5paRjtNrY',
 '#Russia is working on returning embassy to #Libya once security conditions improve https://t.co/IVE09OCDTY',
 'Both Biden and Trump are calling each other “enemies of the state.” Divide and Conquer. Got it? \n(PC: CNN)\n\n#encycloids #trump #biden #potus #lucifer #satan #namecalling #god #divideandconquer https://t.co/aqcnC51vW5',
 'Kadena price soars by 40% after new protocol launches and a major exchange listing\n#russia #gold #cryptocurrency #ethereum\nhttps://t.co/WB1Fl3ZKEL',
 'This article is awesome! Newsom ca

In [12]:
import regex as re
from autocorrect import Speller
import unidecode
from string import punctuation

In [13]:
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemma = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = stopwords.words("english")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def cleantxt(x):
    res = x.lower() #lowercase
    
    res = re.sub(r'<.*?>','',res) #remove html tags
   
    res = re.sub(r'https?://\S+|www\.\S+', '', res) #remove URLs
    
    res = re.sub(r'\d+', '', res) #remove numbers
   
    res = unidecode.unidecode(res) #convert accented characters to ascii characters
    
    emoji_char = re.compile("[" 
                               u"\U0001F600-\U0001F64F"  # emojis
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE) 
    res = emoji_char.sub(r'',res) #remove emojis, pictographs and other unusual characters

    res = re.sub("&amp", "and", res).replace("\n", " ") #remove refuses due to encoding 

    res = res.translate(str.maketrans('', '', punctuation)) #remove punctuation 

    res = re.sub('\s+'," ", res) #remove trailing whitespaces

    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*') #remove stopwords
    res = pattern.sub('', res)

    spell_corrector = Speller(lang='en') #corrector NOTE: this is an english corrector: as a result, some slavic names may be changed
    res = spell_corrector(res)           
    
    return res

    

    

In [15]:
rdd = rdd.map(cleantxt)
rdd.take(50)

['russiancontrolled zaporizhzhia nuclear plant ukraine lost link last remaining main power line un nuclear watchdog said ',
 'russia home best esteemed writers like leo tolstoy floor dostoevsky anton chekhov follow us interesting facts russia russia interesting facts literature artefacts followusnow fp ',
 'russia working returning embassy libya security conditions improve ',
 'biden trump calling enemies state divide conquer got pc cnn encycloids trump biden pots lucifer stan namecalling god divideandconquer ',
 'arena price soars new protocol launches major exchange listing russia gold cryptocurrency ethereal ',
 'article awesome newsok calls gop governors bullies hunter biden huh cnn beyond scraping bottom barrel point biden gop politics vote cadeduelg cathyhobart',
 'noliewithbtc hypocritical statement biden regime fascist',
 'road fascism war ukraine changing europe ',
 'emywinst taylormade rainbowfascisttm fashions something arent biden funnels tax dollars real genocide fascists 

In [16]:
#lemmatization:
rdd = rdd.map(lambda x: lemma.lemmatize(x))
rdd.take(10)

[['russiancontrolled',
  'zaporizhzhia',
  'nuclear',
  'plant',
  'ukraine',
  'lost',
  'link',
  'last',
  'remaining',
  'main',
  'power',
  'line',
  'un',
  'nuclear',
  'watchdog',
  'said',
  ''],
 ['russia',
  'home',
  'best',
  'esteemed',
  'writers',
  'like',
  'leo',
  'tolstoy',
  'floor',
  'dostoevsky',
  'anton',
  'chekhov',
  'follow',
  'us',
  'interesting',
  'facts',
  'russia',
  'russia',
  'interesting',
  'facts',
  'literature',
  'artefacts',
  'followusnow',
  'fp',
  ''],
 ['russia',
  'working',
  'returning',
  'embassy',
  'libya',
  'security',
  'conditions',
  'improve',
  ''],
 ['biden',
  'trump',
  'calling',
  'enemies',
  'state',
  'divide',
  'conquer',
  'got',
  'pc',
  'cnn',
  'encycloids',
  'trump',
  'biden',
  'pots',
  'lucifer',
  'stan',
  'namecalling',
  'god',
  'divideandconquer',
  ''],
 ['arena',
  'price',
  'soars',
  'new',
  'protocol',
  'launches',
  'major',
  'exchange',
  'listing',
  'russia',
  'gold',
  'crypto

In [17]:
rdd = rdd.map(lambda x: x.split(' ')) #now we have our list of baskets (i.e. a list of lists)
rdd.take(10)

Py4JJavaError: ignored